In [ ]:
import os
project_dir = "/Volumes/mnt/Workspace/ai-trading-agents"
os.chdir(project_dir)

In [ ]:
get_ipython().run_line_magic('reload_ext', 'autoreload')  # type: ignore
get_ipython().run_line_magic('autoreload', '2')

In [ ]:
from src import secrets
from src import db

In [ ]:
SECRETS = secrets.load()

ENV = os.environ.get("ENV", "dev")

In [ ]:
await db.connect()

In [ ]:
from datetime import date, timedelta
from src import utils
from src.services.alpaca import (
    get_historical_price_bars,
)

In [ ]:
tickers = ['AAPL', "MSFT"]

In [ ]:
start = (date.today() - timedelta(days=31)).isoformat()
end = date.today().isoformat()

In [ ]:
bars_by_symbol = await get_historical_price_bars(
    symbols=tickers,
    timeframe="1Day",
    start=start,
    end=end,
    sort="desc",  # IMPORTANT: sort by descending timestamp
)

In [ ]:
markdown_tables = []
for ticker in tickers:
    bars = bars_by_symbol[ticker]
    for index, bar in enumerate(bars):
        if index == len(bars) - 1:
            break
        next_price = bars[index+1]['close_price']
        bar['change_percent'] = utils.format_percent(
            (bar['close_price']-next_price) / next_price)

    bars_ = [{
        "date": bar['timestamp'],
        "price": bar['close_price'],
        "change_percent": bar['change_percent']} for bar in bars[:-1]]

    markdown_table = utils.dicts_to_markdown_table(bars_)
    markdown_table = "**" + ticker + "**\n" + markdown_table
    markdown_tables.append(markdown_table)

In [ ]:
markdown_tables_str = "\n".join(markdown_tables)

In [ ]:
stock_price_changes = ("Note: change_percent = (current_price - previos_day_price) / previos_day_price "
                       "Measuing the change percent of the stock price relative to the previous day's price"
                       ) + "\n" + markdown_tables_str

In [ ]:
from markdownify import markdownify as md

html_text = '<h1>Hello, world!</h1><p><strong>This</strong> is a <a href="http://github.com">link</a>.</p>'
markdown_text = md(html_text, strip=['a'])

print(markdown_text)
# Output:
# # Hello, world!
#
# **This** is a [link](http://github.com)